In [8]:
# necessary packages #

#using Pkg
#Pkg.add("Distances")
using Distributions
using Random
using Distances
using LinearAlgebra
using SparseArrays
using IterativeSolvers
using ProgressMeter
using JLD2

In [9]:
include("../../../util2.j")

colnorm (generic function with 1 method)

In [10]:
# Generate simulation data #

Random.seed!(123);
N = 1200 # sample size
Nobs = 1000;
q = 10; K = 50; 

sqrt_Σ_diag = sqrt.([0.5, 1, 0.4, 2, 0.3, 2.5, 3.5, 0.45, 1.5, 0.5]);

β = [1.0 -1.0 1.0 -0.5 2.0 -1.5 0.5 0.3 -2.0 1.5
     -5.0 2.0 3.0 -2.0 -6.0 4.0 5.0 -3.0 6.0 -4.0
     8.0 6.9 -12.0 0.0 -4.0 7.7 -8.8 3.3 6.6 -5.5];

ϕ = rand(Uniform(3 / sqrt(2), 30 / sqrt(2)), K);
Λ = rand(Uniform(-2, 2), K, q); # loading matrix
coords = rand(N, 2);                                          # random location over unit square
ordx = 1:N;
coords_ord = coords[ordx, :];
S_ind = [sample(1:N, Nobs, replace = false, ordered = true) for i in 1:q] 
                                                              # index of observed response for all q responses
S = sort(union([S_ind[i] for i in 1:q]...));                  # location with at least one observation
M_ind = [setdiff(S, S_ind[i]) for i in 1:q];                  # in S not in S1
obs_ind = vcat([S_ind[i] .+ (i - 1) * N for i in 1:q]...)     # index of the observed location for all response among N locations
perm_ind = sortperm(vcat([S_ind[i] for i in 1:q]...))

index_S = zeros(N);                                           # build index indicating which response are observed
index_S_M = fill(0.0, N, q);
for i in 1:q
    index_S_M[S_ind[i], i] .= 1;
    index_S = index_S + 2^(i - 1) * index_S_M[:, i]
end
M_Sind = [findall(x -> x == 0.0, index_S_M[S, i]) for i in 1:q];

In [11]:
# Some data preparations #

m = 10; n = length(S); 
# number of nearest neighbor                       
NN = BuildNN(coords_ord[S, :], m);                            # build nearest neighbor 
nnIndx_col = vcat(NN.nnIndx, 1:n);                            # the index of columns
nnIndx_row = zeros(Int64, 0);                                               
for i in 2:m
    nnIndx_row = vcat(nnIndx_row, fill(i, i-1));
end
nnIndx_row = vcat(nnIndx_row, repeat((m + 1):n, inner = m), 1:n);  # the index of rows


In [12]:
@save "sim3_pars.jld"